In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [3]:
import pandas as pd
import numpy as np

import  matplotlib.pyplot as plt
import pickle

from chemistry import Molecule
from util import score
from pprint import pprint


In [4]:
with open('../data/molecules_enh.pickle', 'rb') as f:
    molecules = pickle.load(f)

In [5]:
structures = pd.read_feather('../data/structures_enh.feather')
molecules_df = pd.read_feather('../data/molecules.feather')
labelled = pd.read_feather('../data/train.feather')
unlabelled = pd.read_feather('../data/test.feather')

In [6]:
labelled_enh = labelled.merge(molecules_df, left_on='molecule_name', right_on='molecule_name')
#labelled_enh.head(32)

In [7]:
unlabelled_enh = unlabelled.merge(molecules_df, left_on='molecule_name', right_on='molecule_name')
#unlabelled_enh.head(32)

In [34]:
from models import SKModel, partition_data

def fn(data_df, n):
    global numeric_df
    data_df = labelled[labelled.type == '1JHC'].head(n)
    #data_df = labelled_enh.head(16)
    train_df, valid_df, test_df = partition_data(data_df, train_frac=1)
    train_df = train_df#.copy()

    model = SKModel(molecules=molecules,
                    structures=structures)
    input_df, numeric_df, output_df = model.setup_data(train_df, train_df)

%lprun -f SKModel.make_atom_columns fn(labelled_enh, 1)

1 0
[ 0.00215042 -0.00603132  0.00197612] [-0.01269814  1.0858041   0.008001  ]
[[-1.2698136e-02  1.0858041e+00  8.0009960e-03]
 [ 2.1504159e-03 -6.0313176e-03  1.9761203e-03]
 [ 1.0117308e+00  1.4637512e+00  2.7657481e-04]
 [-5.4081506e-01  1.4475266e+00 -8.7664372e-01]
 [-5.2381361e-01  1.4379326e+00  9.0639728e-01]]
[0.54597646 0.5459765  1.3739893  1.3740053  1.3740118 ]
[2, 3, 4]
[1.3739893 1.3740053 1.3740118]
Name: dsgdb9nsd_000001
Atoms:
  C 0: [-0.01269814  1.0858041   0.008001  ]
  H 1: [ 0.00215042 -0.00603132  0.00197612]
  H 2: [1.0117308e+00 1.4637512e+00 2.7657481e-04]
  H 3: [-0.54081506  1.4475266  -0.8766437 ]
  H 4: [-0.5238136  1.4379326  0.9063973]
Bonds:
  C(0) - H(1)
  C(0) - H(2)
  C(0) - H(3)
  C(0) - H(4)
Simple field intensity :
  C(0) - 3.354708433151245
  H(1) - 5.975549221038818
  H(2) - 5.975560188293457
  H(3) - 5.975599765777588
  H(4) - 5.975588798522949
Field intensity :
  C(0) - 3.05920657410752e-05
  H(1) - 5.802408695220947
  H(2) - 5.8024206161499

Timer unit: 1e-06 s

Total time: 2.49627 s
File: /home/gautham/work/kaggle/scalar_coupling/notebooks/models.py
Function: make_atom_columns at line 134

Line #      Hits         Time  Per Hit   % Time  Line Contents
   134                                               def make_atom_columns(self, df, atom_syms, atom_index_column, prefix):
   135         3       2312.0    770.7      0.1          atom_sym_column = pd.Categorical(atom_syms, categories=list(self.atom_types) + ['X'])
   136                                           
   137        18         65.0      3.6      0.0          for a in self.atom_types:
   138        15      14266.0    951.1      0.6              df[f'{prefix}_{a}'] = (atom_sym_column == a).astype('int8')
   139         3       4348.0   1449.3      0.2          df[f'{prefix}_weight'] = pd.Series([chemistry.atom_size[atom] for atom in atom_sym_column], index=df.index, dtype='int8')
   140                                           
   141         3       1329.0    44

In [26]:
list(zip(numeric_df.columns,numeric_df.loc[0]))

[('coupling_distance', 1.091952919960022),
 ('coupling_1JHC', 1.0),
 ('atom0_C', 0.0),
 ('atom0_H', 1.0),
 ('atom0_F', 0.0),
 ('atom0_N', 0.0),
 ('atom0_O', 0.0),
 ('atom0_weight', 1.0),
 ('ai', 0.0),
 ('atom0_x', 0.0021504159085452557),
 ('atom0_y', -0.006031317636370659),
 ('atom0_z', 0.0019761202856898308),
 ('atom0_x_mean', -0.012689118273556232),
 ('atom0_y_mean', 1.085796594619751),
 ('atom0_z_mean', 0.008001451380550861),
 ('atom0_dist_to_mean', 1.0919452905654907),
 ('atom0_fi', 5.802408695220947),
 ('atom0_simple_fi', 5.975549221038818),
 ('atomN_C', 1.0),
 ('atomN_H', 0.0),
 ('atomN_F', 0.0),
 ('atomN_N', 0.0),
 ('atomN_O', 0.0),
 ('atomN_weight', 12.0),
 ('atomN_x', -0.012698136270046234),
 ('atomN_y', 1.0858041048049927),
 ('atomN_z', 0.008000995963811874),
 ('atomN_x_mean', -0.012689118273556232),
 ('atomN_y_mean', 1.085796594619751),
 ('atomN_z_mean', 0.008001451380550861),
 ('atomN_dist_to_mean', 1.1744553376047406e-05),
 ('atomN_fi', 3.05920657410752e-05),
 ('atomN_simp